In [3]:
!nvcc -V
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:41:10_Pacific_Daylight_Time_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
Fri Apr  4 11:18:31 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 572.16                 Driver Version: 572.16         CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2070 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A 

In [ ]:
import os
import cv2
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Input, ReLU, Add
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# 모델 정의
def build_advanced_srcnn():
    inputs = Input(shape=(FIXED_SIZE[0], FIXED_SIZE[1], 3))
    x = Conv2D(128, (9, 9), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv2D(64, (5, 5), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv2D(64, (5, 5), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv2D(64, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv2D(64, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv2D(64, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv2D(32, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv2D(3, (3, 3), padding='same')(x)
    outputs = Add()([inputs, x])  # Residual 연결
    
    return Model(inputs, outputs)

# 이미지 전처리 함수 (고정 크기로 리사이즈)
def preprocess_pair_fixed(image_path, upscale_factor=2):
    img = cv2.imread(image_path, cv2.IMREAD_COLOR)
    if img is None:
        raise ValueError(f"이미지를 불러올 수 없습니다: {image_path}")
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, FIXED_SIZE)  # 고정된 크기로 리사이즈

    # 다운샘플 후 업샘플해서 LR 이미지 생성
    lr = cv2.resize(img, (FIXED_SIZE[0] // upscale_factor, FIXED_SIZE[1] // upscale_factor), interpolation=cv2.INTER_CUBIC)
    lr = cv2.resize(lr, FIXED_SIZE, interpolation=cv2.INTER_CUBIC)

    # 정규화
    lr = lr.astype(np.float32) / 255.0
    hr = img.astype(np.float32) / 255.0

    return lr, hr

# Dataset 생성
def create_fixed_dataset(image_dir, batch_size=4):
    image_paths = glob(os.path.join(image_dir, "*.jpg"))
    
    def generator():
        for path in image_paths:
            try:
                lr, hr = preprocess_pair_fixed(path, 10)
                yield lr, hr
            except:
                continue

    dataset = tf.data.Dataset.from_generator(
        generator,
        output_signature=(
            tf.TensorSpec(shape=(FIXED_SIZE[0], FIXED_SIZE[1], 3), dtype=tf.float32),
            tf.TensorSpec(shape=(FIXED_SIZE[0], FIXED_SIZE[1], 3), dtype=tf.float32),
        )
    )
    
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

# 테스트 이미지 전처리 함수
def preprocess_test_image(image_path, upscale_factor=2):
    img = cv2.imread(image_path, cv2.IMREAD_COLOR)
    if img is None:
        raise ValueError("이미지를 불러올 수 없습니다.")
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, FIXED_SIZE)

    # 다운샘플 후 업샘플 → 입력으로 사용
    lr = cv2.resize(img, (FIXED_SIZE[0] // upscale_factor, FIXED_SIZE[1] // upscale_factor), interpolation=cv2.INTER_CUBIC)
    lr_up = cv2.resize(lr, FIXED_SIZE, interpolation=cv2.INTER_CUBIC)

    # 정규화
    lr_up_norm = lr_up.astype(np.float32) / 255.0
    lr_up_norm = np.expand_dims(lr_up_norm, axis=0)  # 배치 차원 추가

    return lr_up_norm, lr, img

# 업스케일링 수행
def upscale_and_show_trt(image_path):
    input_tensor, lr_img, hr_gt = preprocess_test_image(image_path)
    input_tensor = tf.constant(input_tensor)

    # 추론 실행
    output = infer(input_tensor)
    
    # 키 이름이 레이어 이름과 같음 (확인 필요, 예: 'add')
    sr = list(output.values())[0].numpy()[0]
    sr = np.clip(sr * 255.0, 0, 255).astype(np.uint8)

    # 시각화
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 3, 1)
    plt.imshow(lr_img)
    plt.title("Low-Res Input")
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.imshow(sr)
    plt.title("TRT-Optimized Output")
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.imshow(hr_gt)
    plt.title("Original High-Res")
    plt.axis('off')

    plt.show()

# 고정 입력 이미지 크기 설정
FIXED_SIZE = (128, 128)

# 데이터 경로
image_dir = 'img'
dataset = create_fixed_dataset(image_dir)

# 모델 컴파일 & 학습
model = build_advanced_srcnn()
model.compile(optimizer=Adam(learning_rate=1e-4), loss='mean_absolute_error')
model.fit(dataset, epochs=20)

Epoch 1/20
5/5 [==============================] - 10s 2s/step - loss: 0.6103
Epoch 2/20
5/5 [==============================] - 6s 1s/step - loss: 0.4055
Epoch 3/20
5/5 [==============================] - 6s 1s/step - loss: 0.2876
Epoch 4/20
5/5 [==============================] - 7s 1s/step - loss: 0.2166
Epoch 5/20
5/5 [==============================] - 7s 1s/step - loss: 0.1749
Epoch 6/20
5/5 [==============================] - 6s 1s/step - loss: 0.1519
Epoch 7/20
5/5 [==============================] - 6s 1s/step - loss: 0.1371
Epoch 8/20
5/5 [==============================] - 6s 1s/step - loss: 0.1282
Epoch 9/20
5/5 [==============================] - 7s 1s/step - loss: 0.1218
Epoch 10/20
5/5 [==============================] - 7s 1s/step - loss: 0.1175
Epoch 11/20
5/5 [==============================] - 7s 1s/step - loss: 0.1144
Epoch 12/20
5/5 [==============================] - 7s 1s/step - loss: 0.1117
Epoch 13/20
5/5 [==============================] - 7s 2s/step - loss: 0.1096
Epoch 1

INFO:tensorflow:Assets written to: saved_model_srcnn\assets


ERROR:tensorflow:Tensorflow needs to be built with TensorRT support enabled to allow TF-TRT to operate.


ERROR:tensorflow:Tensorflow needs to be built with TensorRT support enabled to allow TF-TRT to operate.


RuntimeError: Tensorflow has not been built with TensorRT support.

In [ ]:
model.save('saved_model', save_format='tf')

from tensorflow.python.compiler.tensorrt import trt_convert as trt

conversion_params = trt.TrtConversionParams(
    precision_mode=trt.TrtPrecisionMode.FP16,
    max_workspace_size_bytes=1 << 25
)

converter = trt.TrtGraphConverterV2(
    input_saved_model_dir="saved_model",
    conversion_params=conversion_params
)

converter.convert()
converter.save("trt_saved_model")

trt_model = tf.saved_model.load("trt_saved_model")
infer = trt_model.signatures["serving_default"]

upscale_and_show_trt('image.jpg')

INFO:tensorflow:Assets written to: saved_model_srcnn\assets


INFO:tensorflow:Assets written to: saved_model_srcnn\assets


ERROR:tensorflow:Tensorflow needs to be built with TensorRT support enabled to allow TF-TRT to operate.


ERROR:tensorflow:Tensorflow needs to be built with TensorRT support enabled to allow TF-TRT to operate.


RuntimeError: Tensorflow has not been built with TensorRT support.

In [ ]:
import tensorflow as tf
from tensorflow.python.compiler.tensorrt import trt_convert as trt

try:
    converter = trt.TrtGraphConverterV2(
        input_saved_model_dir="saved_model_srcnn"
    )
    converter.convert()
    print("TensorRT 변환 성공")
except Exception as e:
    print("TensorRT 변환 실패")
    print("에러 메시지:", e)

ERROR:tensorflow:Tensorflow needs to be built with TensorRT support enabled to allow TF-TRT to operate.


ERROR:tensorflow:Tensorflow needs to be built with TensorRT support enabled to allow TF-TRT to operate.


TensorRT 변환 실패 ❌
에러 메시지: Tensorflow has not been built with TensorRT support.
